# XGBoost Model Evaluation

This notebook loads the preprocessed data, retrains the XGBoost model (for self-containment of evaluation), and then evaluates its performance using various metrics, including ROC analysis and threshold tuning for specific class optimization.

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_curve, auc, roc_auc_score, precision_recall_fscore_support
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
# Define input path
input_parquet_path = 'data/preprocessed_data.parquet'
# Load the preprocessed data
print(f"Loading preprocessed data from {input_parquet_path}...")
try:
    df = pd.read_parquet(input_parquet_path)
    print("Data loaded successfully.")
    df.info()
except FileNotFoundError:
    print(f"Error: File not found at {input_parquet_path}. Please ensure '1_consolidate_data.ipynb' has been run.")
except Exception as e:
    print(f"\nAn error occurred while loading the Parquet file: {e}")

In [ ]:
# Separate train and test sets based on the 'split' column
train_df = df[df['split'] == 'train']
test_df = df[df['split'] == 'test']

# Separate features (X) and target (y)
X_train_scaled = train_df.drop(['Class', 'split'], axis=1)
y_train = train_df['Class']

X_test_scaled = test_df.drop(['Class', 'split'], axis=1)
y_test = test_df['Class']

print(f"Training features shape: {X_train_scaled.shape}, Training target shape: {y_train.shape}")
print(f"Test features shape: {X_test_scaled.shape}, Test target shape: {y_test.shape}")

In [ ]:
# Convert target variable 'Class' from object ('n'/'y') to numeric (0/1) if necessary
if y_train.dtype == 'object':
    print("\nConverting target variable 'Class' to numeric (n=0, y=1)...")
    y_train = y_train.map({'n': 0, 'y': 1})
    y_test = y_test.map({'n': 0, 'y': 1})
    print("Target variable converted.")
else:
    print("\nTarget variable 'Class' is already numeric or in an unexpected format.")

In [ ]:
# Calculate class distribution in the training set for scale_pos_weight
scale_pos_weight_val = 1 # Default
if y_train.dtype == 'int64' or y_train.dtype == 'int32':
    count_class_0 = (y_train == 0).sum()
    count_class_1 = (y_train == 1).sum()
    print(f"\nTraining data class distribution: Class 0 (n): {count_class_0}, Class 1 (y): {count_class_1}")
    if count_class_1 > 0:
        scale_pos_weight_val = count_class_0 / count_class_1
        print(f"Calculated scale_pos_weight: {scale_pos_weight_val:.4f}")
    else:
        print("Warning: No positive class (1) instances in y_train. scale_pos_weight set to 1.")
else:
    print(f"Warning: y_train is not numeric (dtype: {y_train.dtype}). scale_pos_weight set to 1.")

In [ ]:
# Define and Train XGBoost Classifier
xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False, # Recommended to avoid warnings
    random_state=42,
    n_estimators=100, # Default, can be tuned
    scale_pos_weight=scale_pos_weight_val
)

print("\nTraining XGBoost model for evaluation notebook...")
xgb_clf.fit(X_train_scaled, y_train)
print("Model training complete.")

In [ ]:
# Make Predictions on the test set
y_pred_xgb = xgb_clf.predict(X_test_scaled)
print("Predictions made on the test set.")

In [ ]:
def evaluate_model(y_true, y_pred, X_features_for_proba, model, model_name):
    """Calculates, prints, and plots evaluation metrics for a binary classifier."""
    print(f"\n--- {model_name} Evaluation ---")
    
    # Accuracy
    accuracy = accuracy_score(y_true, y_pred)
    print(f"Accuracy: {accuracy:.4f}")
    
    # Classification Report
    print("\nClassification Report:")
    target_names = ['Class n (0)', 'Class y (1)'] if np.all(np.isin(y_true.unique(), [0, 1])) else None
    print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))
    
    # Confusion Matrix
    print("\nConfusion Matrix:")
    cm = confusion_matrix(y_true, y_pred)
    print(cm)
    
    # Plot Confusion Matrix
    plt.figure(figsize=(6, 4))
    classes = model.classes_ if hasattr(model, 'classes_') else [0, 1]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Greens', 
                xticklabels=classes, 
                yticklabels=classes)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix - {model_name}')
    plt.show()

    # ROC Curve and AUC
    if hasattr(model, "predict_proba") and X_features_for_proba is not None:
        y_pred_proba = model.predict_proba(X_features_for_proba)[:, 1]
        fpr, tpr, thresholds_roc = roc_curve(y_true, y_pred_proba)
        roc_auc = auc(fpr, tpr)
        print(f"\nROC AUC Score: {roc_auc:.4f}")

        plt.figure(figsize=(8, 6))
        plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'Receiver Operating Characteristic (ROC) - {model_name}')
        plt.legend(loc="lower right")
        plt.show()
    else:
        print("\nROC Curve not available: Model lacks predict_proba or features for probabilities not provided.")

In [ ]:
# Evaluate the XGBoost model
evaluate_model(y_test, y_pred_xgb, X_test_scaled, xgb_clf, "XGBoost")

## Threshold Tuning Analysis for Class n (0)

This section explores how changing the classification threshold affects precision, recall, and F1-score, specifically for identifying 'Class n (0)' correctly.

In [ ]:
# Define a range of threshold values to test
threshold_values = np.arange(0.05, 1.0, 0.05)

# Get predicted probabilities for the positive class (Class 1, 'y') from the trained xgb_clf
y_pred_proba_for_thresholding = xgb_clf.predict_proba(X_test_scaled)[:, 1]

precisions_class0 = []
recalls_class0 = []
f1s_class0 = []

print("\n--- Threshold Tuning for Class n (0) (Uncalibrated Model) ---")
print(f"{'Threshold':<10} | {'Precision (0)':<15} | {'Recall (0)':<12} | {'F1-score (0)':<12} | {'TP (0)':<7} | {'FP (0)':<7} | {'FN (0)':<7}")
print("-" * 80)

for thresh in threshold_values:
    # If prob_for_class_1 >= threshold, predict 1 (y), else 0 (n)
    y_pred_at_threshold = (y_pred_proba_for_thresholding >= thresh).astype(int)
    
    # Calculate metrics for Class 0 (label 0)
    p, r, f, s = precision_recall_fscore_support(y_test, y_pred_at_threshold, labels=[0, 1], zero_division=0)
    
    # Confusion matrix for this threshold to get TP, FP, FN for class 0
    cm_thresh = confusion_matrix(y_test, y_pred_at_threshold, labels=[0,1])
    # For class 0: TP(0) = cm_thresh[0,0], FP(0) = cm_thresh[1,0] (predicted 0, but was 1), FN(0) = cm_thresh[0,1] (predicted 1, but was 0)
    tp_c0 = cm_thresh[0,0] if cm_thresh.shape == (2,2) else 0
    fp_c0 = cm_thresh[1,0] if cm_thresh.shape == (2,2) else 0 
    fn_c0 = cm_thresh[0,1] if cm_thresh.shape == (2,2) else 0

    precisions_class0.append(p[0])
    recalls_class0.append(r[0])
    f1s_class0.append(f[0])
    
    print(f"{thresh:<10.2f} | {p[0]:<15.4f} | {r[0]:<12.4f} | {f[0]:<12.4f} | {tp_c0:<7} | {fp_c0:<7} | {fn_c0:<7}")

# Plotting the metrics
plt.figure(figsize=(12, 7))
plt.plot(threshold_values, precisions_class0, label='Precision (Class 0)', marker='o')
plt.plot(threshold_values, recalls_class0, label='Recall (Class 0)', marker='x')
plt.plot(threshold_values, f1s_class0, label='F1-score (Class 0)', marker='s')
plt.title('Precision, Recall, and F1-score for Class n (0) vs. Threshold (Uncalibrated Model)')
plt.xlabel('Threshold (Probability for Class y (1))')
plt.ylabel('Score')
plt.xticks(np.round(threshold_values,2))
plt.legend()
plt.grid(True)
plt.show()

# Find threshold that maximizes F1-score for Class 0
if f1s_class0 and not all(v == 0 for v in f1s_class0): # Check if list is not empty and contains non-zero values
    optimal_idx_f1_class0 = np.argmax(f1s_class0)
    optimal_threshold_f1_class0 = threshold_values[optimal_idx_f1_class0]
    print(f"\nOptimal threshold for maximizing F1-score for Class n (0) (Uncalibrated Model): {optimal_threshold_f1_class0:.2f}")
    print(f"  Precision (Class 0) at this threshold: {precisions_class0[optimal_idx_f1_class0]:.4f}")
    print(f"  Recall (Class 0) at this threshold: {recalls_class0[optimal_idx_f1_class0]:.4f}")
    print(f"  F1-score (Class 0) at this threshold: {f1s_class0[optimal_idx_f1_class0]:.4f}")
else:
    print("\nCould not determine optimal threshold for F1-score (Class 0) (Uncalibrated Model) as no valid F1 scores were calculated or all were zero.")

In [ ]:
# ## Feature Importance Analysis
#
# This section visualizes the importance of each feature in the XGBoost model.

# Get feature importances from the trained XGBoost model
importances = xgb_clf.feature_importances_
feature_names = X_train_scaled.columns # X_train_scaled has the feature names

# Create a DataFrame for better visualization
feature_importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})

# Sort features by importance
feature_importance_df = feature_importance_df.sort_values(by='importance', ascending=False)

print("\n--- Feature Importance Analysis ---")
print("Top 10 Feature Importances:")
print(feature_importance_df.head(10))
print("\n") # Add a newline for better separation in output

# Plotting the feature importances (e.g., top 20 features)
plt.figure(figsize=(12, 8)) # Adjust figure size as needed
num_features_to_plot = min(len(feature_importance_df), 20) # Plot top 20 or fewer if not available
sns.barplot(x='importance', y='feature', data=feature_importance_df.head(num_features_to_plot), palette='viridis')
plt.title(f'Top {num_features_to_plot} Feature Importances - XGBoost')
plt.xlabel('Importance Score')
plt.ylabel('Features')
plt.tight_layout() # Adjust layout to prevent labels from overlapping
plt.show()

## SHAP (SHapley Additive exPlanations) Value Analysis

SHAP values provide a way to explain the output of machine learning models by quantifying the contribution of each feature to a particular prediction. This offers more granular insights than global feature importance.

- **Summary Plot:** Shows the distribution of SHAP values for each feature, indicating not only the importance but also the direction of the relationship (e.g., whether high values of a feature increase or decrease the prediction).
- **Force Plot (for individual predictions):** Illustrates how features contributed to pushing a single prediction away from the base value (average prediction over the training set).

In [ ]:
# Note: If you haven't installed shap, you might need to run: !pip install shap
try:
    import shap
    shap_available = True
    shap.initjs() # Initialize JavaScript visualization in the notebook
except ImportError:
    print("SHAP library not found. Please install it to run this section (e.g., pip install shap).")
    shap_available = False

if shap_available:
    print("\n--- SHAP Value Analysis ---")
    
    # Create a SHAP TreeExplainer for the XGBoost model
    # For tree-based models like XGBoost, TreeExplainer is more efficient.
    explainer = shap.TreeExplainer(xgb_clf)
    
    # Calculate SHAP values for the test set
    # This can take a moment for larger datasets
    print("Calculating SHAP values for the test set...")
    shap_values = explainer.shap_values(X_test_scaled)
    print("SHAP values calculated.")
    
    # SHAP Summary Plot (Beeswarm)
    print("\nSHAP Summary Plot (Beeswarm):")
    shap.summary_plot(shap_values, X_test_scaled, plot_type="beeswarm")
    
    # SHAP Force Plot for the first instance in the test set
    if len(X_test_scaled) > 0:
        print("\nSHAP Force Plot for the first test instance:")
        # For binary classification, shap_values can be an array or a list of two arrays (one for each class).
        # If it's a list, explainer.expected_value might also be a list.
        # We typically explain the probability of the positive class.
        expected_value_to_use = explainer.expected_value
        shap_values_to_use = shap_values
        
        if isinstance(explainer.expected_value, (list, np.ndarray)) and len(explainer.expected_value) == 2:
            # Common case for binary classifiers from XGBoost when explain_output='probability'
            expected_value_to_use = explainer.expected_value[1] # for the positive class (class 1)
            shap_values_to_use = shap_values[1] # SHAP values for the positive class
            
        # If shap_values is a 2D array (instances, features), it's likely for the positive class already
        # or for models where explainer.expected_value is a single value.
        
        shap.force_plot(expected_value_to_use, 
                        shap_values_to_use[0,:], 
                        X_test_scaled.iloc[0,:], 
                        matplotlib=True) # Use matplotlib for better rendering in some environments
    else:
        print("Test set is empty, cannot generate force plot for an instance.")


## Cross-Validation Evaluation

To assess the model's generalization performance more robustly and ensure it's not overfitting to the specific train-test split, we perform k-fold cross-validation on the training data. We'll use Stratified K-Fold to maintain class proportions in each fold.


In [ ]:

from sklearn.model_selection import cross_val_score, StratifiedKFold
# import numpy as np # Already imported

# Re-define the XGBoost classifier for cross-validation to ensure fresh state for each fold
# Use the same parameters as the main model
cv_xgb_clf = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False, 
    random_state=42,
    n_estimators=100, # Or use tuned n_estimators if available from a CV tuning step
    scale_pos_weight=scale_pos_weight_val # Use the calculated scale_pos_weight
)

# Define Stratified K-Fold
n_splits_cv = 5 # Number of folds
strat_k_fold = StratifiedKFold(n_splits=n_splits_cv, shuffle=True, random_state=42)

print(f"\n--- Cross-Validation Evaluation ({n_splits_cv}-fold) ---")

# Perform cross-validation for different metrics
# Note: X_train_scaled and y_train are used here
scoring_metrics = {
    'accuracy': 'accuracy',
    'roc_auc': 'roc_auc',
    'f1': 'f1_weighted' # Using f1_weighted for potentially imbalanced classes
}

cv_results = {}
for metric_name, scorer in scoring_metrics.items():
    try:
        scores = cross_val_score(cv_xgb_clf, X_train_scaled, y_train, cv=strat_k_fold, scoring=scorer)
        cv_results[metric_name] = scores
        print(f"Cross-validation {metric_name.upper()} scores: {scores}")
        print(f"Mean {metric_name.upper()}: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")
    except Exception as e:
        print(f"Could not calculate cross-validation {metric_name.upper()}. Error: {e}")
    print("-" * 30)

# You can further analyze or store cv_results if needed
